In [1]:
import Pkg
Pkg.add("Flux")
Pkg.add("NCDatasets")
Pkg.add("TSVD")
Pkg.add("Statistics")
Pkg.add("Compat")
Pkg.add("LinearAlgebra")
Pkg.add("Glob")
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Distributions")
Pkg.add("ProgressMeter")
Pkg.add("PyPlot")
Pkg.add("Random")
using Flux
using Flux: train!
using Plots
using TSVD
using Statistics
using LinearAlgebra
using Compat
using Glob
using NCDatasets
using CSV
using DataFrames
using Distributions: Categorical, Dirichlet
using ProgressMeter
using PyPlot
using Random

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Ch

## Load the training data

In [42]:
training_files = sort(glob("../tests/training_data/*.nc"))

nf = length(training_files)
d = NCDataset(training_files[1], "r")
v = d["velsurf_mag"]
nx, ny, nt = size(v)

Data = zeros(nx*ny, nf * nt)
ids = zeros(Int64, nf)
for (k, training_file) in enumerate(training_files)
    print("Reading  ", training_file, "\n");
    m_id = match(r"id_(.+?)_", training_file)
    ids[k] = parse(Int, m_id[1])
    d = NCDataset(training_file, "r")
    v = d["velsurf_mag"][:]
    v = nomissing(v, 0.0)
    data = reshape(v, ny * nx, nt)
    Data[:, (k - 1 ) * nt + 1:k * nt] = data
end


# obs_file ="../data/observed_speeds/greenland_vel_mosaic250_v1_g9000m.nc"
# obs = ncread(obs_file, "velsurf_mag")
# Obs = reshape(obs, ny*nx)
# nx_obs, ny_obs = size(obs)

Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_0_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_100_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_101_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_102_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_103_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_104_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_105_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_106_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_107_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_108_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_109_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_10_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_110_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_111_0_50.nc
Reading  

Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_229_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_22_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_230_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_231_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_232_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_233_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_234_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_235_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_236_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_237_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_238_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_239_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_23_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_240_0_50.nc
Reading 

Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_359_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_35_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_360_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_361_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_362_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_363_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_364_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_365_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_366_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_367_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_368_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_369_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_36_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_370_0_50.nc
Reading 

Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_485_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_486_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_487_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_488_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_489_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_48_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_490_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_491_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_492_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_493_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_494_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_495_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_496_0_50.nc
Reading  ../tests/training_data/velsurf_mag_gris_g9000m_id_497_0_50.nc
Reading

## Read training samples

In [43]:
X_df = DataFrame(CSV.File("../data/samples/velocity_calibration_samples_50.csv"))
X_df = X_df[ [x in ids for x in X_df[!, :id]] ,:]
X = transpose(Matrix(X_df[!, 2:9]))
X_mean = mean(X, dims=1);
X_std = std(X, dims=1);
X_scaled = (X .- X_mean) ./ X_std;
X_train = X_scaled;
n_parameters, n_samples = size(X);

## Data preprocessing

Log10-transform the training data and set -Inf to 0

In [44]:
F = log10.(Data)
F = replace!(F, -Inf=>0)
n_grid_points, _ = size(F)
dirichlet_dist = Dirichlet(n_samples, 1)

area = ones(n_grid_points);
area = area ./ sum(area);

# Number of eigenglaciers
q = 50;

## Function to get Eigenglaciers using SVD

In [56]:
function get_eigenglaciers(omegas, F, q)
    
    F_mean = sum(F .* omegas, dims=2);
    F_bar = F .- F_mean;

    Z = diagm(sqrt.(omegas[1, :] * n_grid_points))
    U, S, V = tsvd(Z * transpose(F_bar), q);
    lamda = S.^2 / n_grid_points
    V_hat = V * diagm(sqrt.(lamda));
    
    return V_hat, F_bar, F_mean
end;

## Set up the Neural Network

In [57]:
n_hidden = 128

struct NNModel
  chain::Chain
end

function (m::NNModel)(x, add_mean=false)
    if add_mean
        return V_hat * m.chain(x) .+ F_mean
    else
        return V_hat * m.chain(x)
    end

end


# Call @functor to allow for training. Described below in more detail.
Flux.@functor NNModel

chain = Chain(
    Dense(n_parameters => n_hidden),
    LayerNorm(n_hidden),
    Dropout(0.0),
    Dense(n_hidden => n_hidden),
    LayerNorm(n_hidden),
    Dropout(0.5),
    Dense(n_hidden => n_hidden),
    LayerNorm(n_hidden),
    Dropout(0.5),
    Dense(n_hidden => n_hidden),
    LayerNorm(n_hidden),
    Dropout(0.5),
    Dense(n_hidden => q),
    );

In [58]:
no_models = 2
n_epochs = 11
opt = Adam(0.1, (0.9, 0.8));

## Loss function

In [59]:
loss(y_pred, y, o) = sum(sum(abs.((y_pred - y)).^2 .* area, dims=1) .* o);

## We're ready for training

In [65]:
models = []
for model_index in 1:no_models
    println("Training surrogate model ", model_index)
    Random.seed!(model_index)
    omegas = transpose(rand(dirichlet_dist, 1))
    omegas_0 = omegas ./ size(omegas)[1];
    
    V_hat, F_bar, F_mean = get_eigenglaciers(omegas, F, q)
    train_loader = Flux.DataLoader((X_train, F_bar, omegas), batchsize = 128, shuffle = true)
    model = NNModel(chain);
    ps = Flux.params(model);
    opt_state = Flux.setup(opt, model);
    
    println("  epoch, train_loss, test_loss")
    for epoch in 1:n_epochs
        for (x, y, o) in train_loader

          # Calculate the gradient of the objective
          # with respect to the parameters within the model:
          grads = Flux.gradient(model) do m
              y_pred = m(x)
              loss(y_pred, y, o)
          end

          # Update the parameters so as to reduce the objective,
          # according the chosen optimisation rule:
          Flux.update!(opt_state, model, grads[1])
        end
        F_pred = model(X_scaled)
        train_loss = loss(F_pred, F_bar, omegas)
        test_loss = loss(F_pred, F_bar, omegas_0)
        if epoch % 5 == 0
            println("  ", epoch, " ", train_loss, " ", test_loss)
        end
    end
    push!(models, model)
end

Training surrogate model 1
  epoch, train_loss, test_loss
  5 0.03409823949278852 0.03409823949278852
  10 0.027388582027968927 0.027388582027968927
Training surrogate model 2
  epoch, train_loss, test_loss
  5 0.02841099847170035 0.02841099847170035
  10 0.028101036558572788 0.028101036558572788


In [66]:
n_glaciers = 10
p = ones(n_samples)
p = p / sum(p)
# This does sampling with replacement, need to figure out how to do
# sampling without replacement
P = Categorical(p)
glaciers = rand(P, n_glaciers);

## Now calcuate some metrics to assess the surrogate committee

In [68]:
F_train = F
maes = []
for m in glaciers
    for (model_index, model) in enumerate(models)
        X_val = X_train[:, m]
        Y_val = F_train[:, m]
        Y_pred = model(X_val, true)
        mae = Flux.mae(10 .^ mean(Y_pred, dims=2), 10 .^ mean(Y_val, dims=2))
        push!(maes, mae)
    end
end
mae = mean(maes)
print("MAE: ", mae)

MAE: 16.403470969124847